
# 🌳 Árboles de Decisión — Versión Explicativa (v3)  
**Curso de Análisis de Datos con IA**  
**Autores:** Doribel Rodríguez y Antonio Vantaggiato

**Objetivo pedagógico.** Comprender y aplicar árboles de decisión con dos casos:
1) **Dataset clásico Iris** (comparación técnica).  
2) **Caso educativo realista** (predicción de aprobación).  

Incluye explicación de **matriz de confusión**, métricas (accuracy, precision, recall, F1) e **importancias de variables**.



## 🧠 Conceptos clave
- Un **árbol de decisión** divide el espacio de características con reglas *if/else* para asignar una clase en sus **hojas**.  
- **Criterios**: *Gini* y *Entropía* miden la pureza de las particiones.  
- **Sobreajuste**: árboles muy profundos memorizan el entrenamiento; control con `max_depth`, `min_samples_leaf`, `ccp_alpha` (poda).

### 📊 Matriz de confusión (clasificación binaria)
| Real \ Predicho | Positivo | Negativo |
|---:|:---:|:---:|
| **Positivo** | **TP** | **FN** |
| **Negativo** | **FP** | **TN** |

- **TP**: acierto positivo; **TN**: acierto negativo.  
- **FP**: falso positivo; **FN**: falso negativo.  

Métricas derivadas:  
- **Accuracy** = (TP+TN)/Total  
- **Precision** = TP/(TP+FP)  
- **Recall** = TP/(TP+FN)  
- **F1** = 2·(Precision·Recall)/(Precision+Recall)


## 1) Preparación del entorno

In [ ]:

import numpy as np
import pandas as pd
import matplotlib.pyplot as plt

from sklearn.model_selection import train_test_split, cross_val_score
from sklearn.tree import DecisionTreeClassifier, plot_tree
from sklearn.metrics import (
    accuracy_score, precision_score, recall_score, f1_score,
    confusion_matrix, ConfusionMatrixDisplay, classification_report
)
from sklearn import datasets

%matplotlib inline



## 2) Caso A — Dataset clásico **Iris**
Entrenamos un árbol base y visualizamos el desempeño.


In [ ]:

iris = datasets.load_iris()
X = iris.data
y = iris.target
feature_names = iris.feature_names
target_names = iris.target_names

X_train, X_test, y_train, y_test = train_test_split(
    X, y, test_size=0.25, random_state=42, stratify=y
)

arbol_iris = DecisionTreeClassifier(random_state=42)
arbol_iris.fit(X_train, y_train)

y_pred = arbol_iris.predict(X_test)
acc = accuracy_score(y_test, y_pred)

print(f"Exactitud (accuracy) en test: {acc:.3f}")
cm = confusion_matrix(y_test, y_pred, labels=arbol_iris.classes_)
disp = ConfusionMatrixDisplay(confusion_matrix=cm, display_labels=target_names[arbol_iris.classes_])
disp.plot(cmap="Blues")
plt.title("Matriz de confusión — Iris")
plt.show()

plt.figure(figsize=(12,7))
plot_tree(arbol_iris, feature_names=feature_names, class_names=target_names, filled=True, rounded=True)
plt.title("Árbol de decisión — Iris")
plt.show()


### 2.1) Profundidad máxima y validación cruzada

In [ ]:

depths = range(1, 11)
scores = []
for d in depths:
    clf = DecisionTreeClassifier(max_depth=d, random_state=42)
    cv = cross_val_score(clf, X, y, cv=5)
    scores.append(cv.mean())

plt.figure(figsize=(7,4))
plt.plot(depths, scores, marker="o")
plt.xlabel("max_depth")
plt.ylabel("Accuracy CV (5-fold)")
plt.title("Elección de profundidad — Iris")
plt.grid(True, linestyle="--", alpha=0.5)
plt.show()



## 3) Caso B — **Aprobación de estudiantes** (simulado realista)
Variables:
- **asistencia_%**, **horas_estudio**, **participa_clase** (0/1), **tareas_entregadas** (0–10),  
  **promedio_prev** (0–100), **uso_plataforma** (0–1), **nivel_estres** (0–10), **satisfaccion_curso** (0–10).

Objetivo: **aprobado** (0/1). Generamos la etiqueta con una regla latente probabilística para acercarnos a la realidad.


In [ ]:

rng = np.random.default_rng(7)
n = 400

asistencia = rng.normal(82, 10, n).clip(40, 100)
horas = rng.normal(6, 2.2, n).clip(0, 14)
participa = rng.integers(0, 2, n)
tareas = rng.integers(0, 11, n)
prom_prev = rng.normal(74, 13, n).clip(0, 100)
uso_plataforma = rng.beta(2, 1.5, n)  # 0-1
estres = rng.normal(5.5, 2.0, n).clip(0, 10)
satisfaccion = rng.normal(7.2, 1.8, n).clip(0, 10)

logit = (
    -7.0
    + 0.035*asistencia
    + 0.30*horas
    + 0.55*participa
    + 0.12*tareas
    + 0.028*prom_prev
    + 0.9*uso_plataforma
    - 0.18*estres
    + 0.22*satisfaccion
)
prob = 1 / (1 + np.exp(-logit))
aprobado = (rng.uniform(0,1,n) < prob).astype(int)

df = pd.DataFrame({
    "asistencia_%": asistencia.round(1),
    "horas_estudio": horas.round(1),
    "participa_clase": participa,
    "tareas_entregadas": tareas,
    "promedio_prev": prom_prev.round(1),
    "uso_plataforma": uso_plataforma.round(3),
    "nivel_estres": estres.round(1),
    "satisfaccion_curso": satisfaccion.round(1),
    "aprobado": aprobado
})
df.head()


### 3.1) Entrenamiento, matriz de confusión y métricas

In [ ]:

X = df.drop(columns=["aprobado"])
y = df["aprobado"]

X_train, X_test, y_train, y_test = train_test_split(
    X, y, test_size=0.25, random_state=42, stratify=y
)

tree = DecisionTreeClassifier(random_state=42, max_depth=5, min_samples_leaf=5)
tree.fit(X_train, y_train)

y_pred = tree.predict(X_test)

acc = accuracy_score(y_test, y_pred)
prec = precision_score(y_test, y_pred, zero_division=0)
rec = recall_score(y_test, y_pred, zero_division=0)
f1 = f1_score(y_test, y_pred, zero_division=0)

print(f"Accuracy:  {acc:.3f}")
print(f"Precision: {prec:.3f}")
print(f"Recall:    {rec:.3f}")
print(f"F1-score:  {f1:.3f}")

cm = confusion_matrix(y_test, y_pred, labels=tree.classes_)
disp = ConfusionMatrixDisplay(confusion_matrix=cm, display_labels=tree.classes_)
disp.plot(cmap="Greens")
plt.title("Matriz de confusión — Estudiantes (0=No, 1=Sí)")
plt.show()

print("\nReporte de clasificación:\n")
print(classification_report(y_test, y_pred, digits=3))


### 3.2) Importancia de variables y estructura del árbol

In [ ]:

importancias = pd.Series(tree.feature_importances_, index=X.columns).sort_values(ascending=False)
display(importancias)

plt.figure(figsize=(7,4))
importancias.plot(kind="bar")
plt.title("Importancia de características — Árbol (estudiantes)")
plt.ylabel("Importancia")
plt.grid(axis="y", linestyle="--", alpha=0.5)
plt.show()

plt.figure(figsize=(14,8))
plot_tree(tree, feature_names=X.columns, class_names=["No","Sí"], filled=True, rounded=True)
plt.title("Árbol de decisión — Aprobación de estudiantes")
plt.show()



## 4) Interpretación responsable (cierre)
- Revisa la matriz de confusión: ¿hay más **falsos negativos** o **falsos positivos**?  
  En contexto educativo, priorizar *recall* puede ser deseable para **no pasar por alto** estudiantes en riesgo.
- Las **importancias** sugieren qué variables guían la decisión (p. ej., horas de estudio, tareas, satisfacción).  
  No confundas **correlación** con **causalidad**: usa estos hallazgos como **insumos**, no veredictos.
- Controla la **complejidad** con `max_depth`/`min_samples_leaf` y valida con datos independientes.
